# Variability of the surface elevation change: Timeseries

## Import packages

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats
import xarray as xr
from c3s_eqc_automatic_quality_control import download

plt.style.use("seaborn-v0_8-notebook")

## Define parameters

In [ ]:
regions = {
    "Greenland": {
        "domain": "greenland",
        "x_slice": slice(None, None),
        "y_slice": slice(None, None),
    },
    "Antarctica": {
        "domain": "antarctica",
        "x_slice": slice(None, None),
        "y_slice": slice(None, None),
    },
    "Jakobshavn Glacier": {
        "domain": "greenland",
        "x_slice": slice(26, 27),
        "y_slice": slice(48, 49),
    },
}

## Define request

In [ ]:
collection_id = "satellite-ice-sheet-elevation-change"
request = {
    "variable": "all",
    "format": "zip",
    "climate_data_record_type": "icdr",
    "version": "3_0",
}

## Function to cache

In [ ]:
def get_surface_elevation_change_and_missing_data(ds, x_slice, y_slice):
    ds = ds.isel(x=x_slice, y=y_slice)
    ds["time"].attrs["long_name"] = "Time"

    (sec_name,) = set(ds.data_vars) & {"sec", "dhdt"}
    da = ds[sec_name]
    da.attrs["long_name"] = "Surface elevation change"

    da_err = ds[f"{sec_name}_uncert"]
    da_err.attrs["long_name"] = "Surface elevation change standard error"

    (mask_name,) = set(ds.data_vars) & {"land_mask", "surface_type"}
    mask = ds[mask_name] > 0
    missing = 100 * (mask.sum() - da.notnull().sum(("x", "y"))) / mask.sum()
    missing.attrs = {"long_name": "Missing data", "units": "%"}

    ds = xr.merge(
        [da.rename("sec"), da_err.rename("sec_err"), missing.rename("missing")]
    )
    return ds.mean(("x", "y"), keep_attrs=True)

## Download and transform data

In [ ]:
datasets = {}
for region, kwargs in regions.items():
    print(f"{region=}")
    datasets[region] = download.download_and_transform(
        collection_id,
        request | {"domain": kwargs["domain"]},
        transform_func=get_surface_elevation_change_and_missing_data,
        transform_func_kwargs={k: v for k, v in kwargs.items() if k != "domain"},
    ).compute()


pixel_datasets = {
    k: v for k, v in datasets.items() if k not in ("Greenland", "Antarctica")
}
datasets = {k: v for k, v in datasets.items() if k not in pixel_datasets}

## Define plotting functions

In [ ]:
def plot_timeseries(datasets, varname):
    fig, axs = plt.subplots(len(datasets), 1, layout="constrained", squeeze=False)
    for ax, (region, ds) in zip(axs.flatten(), datasets.items()):
        da = ds[varname]
        da["time"].attrs["units"] = "yr"
        title = f"{region} Ice Sheet"
        print(
            f"Average {title} {da.attrs['long_name'].lower()}: "
            f"{da.mean().values:.4f} {da.attrs['units']}"
        )
        da.plot(ax=ax)
        ax.set_title(title)
        ax.grid()
    print()
    return fig, axs


def plot_trends(datasets, varname):
    year_to_ns = 1.0e9 * 60 * 60 * 24 * 365

    fig, axs = plt.subplots(len(datasets), 1, layout="constrained", squeeze=False)
    for ax, (region, ds) in zip(axs.flatten(), datasets.items()):
        ds["time"].attrs["units"] = "yr"
        with xr.set_options(keep_attrs=True):
            da = ds[varname].cumsum("time") / 12
        da.attrs["units"] = da.attrs["units"].split("/", 1)[0]
        print(f"{region}:")
        for label, degree in zip(
            (
                "Linear",
                "Quadratic",
            ),
            (1, 2),
        ):
            # Compute coefficients
            coeffs = da.polyfit("time", degree)["polyfit_coefficients"]

            # Plot trends and print stats
            equation = []
            for deg, coeff in coeffs.groupby("degree"):
                coeff = coeff.squeeze() * (year_to_ns**deg)
                if deg == degree:
                    if deg == 1:
                        quantity = "slope"
                        units = f"{da.attrs['units']}/yr"
                    elif deg == 2:
                        quantity = "acceleration"
                        units = f"{da.attrs['units']}/yr^2"
                    else:
                        raise ValueError(f"{deg=}")
                    print(
                        f"\tThe {quantity} of the Ice Sheet {da.attrs['long_name']} "
                        f"is {degree*coeff:.3f} {units}."
                    )
                    if deg == 1:
                        _, p_value = scipy.stats.kendalltau(da["time"], da)
                        s_lev = 0.05
                        is_significant = p_value < s_lev
                        print(
                            " ".join(
                                [
                                    "\tThe trend",
                                    "is significant"
                                    if is_significant
                                    else "is not significant",
                                    f"at an alpha level of {s_lev}, i.e. a monotonic trend",
                                    "is present."
                                    if is_significant
                                    else "is not present.",
                                ]
                            )
                        )
                equation.append(
                    f"{float(coeff):+.3f}{'x' if deg else ''}{f'$^{deg}$' if deg>1 else ''}"
                )
            label = f"{label}: {' '.join(equation[::-1])}"
            fit = xr.polyval(da["time"], coeffs)
            fit.plot(label=label, ax=ax)
        da.plot(label="Data", ax=ax)
        ax.set_title(f"{region} Ice Sheet")
        ax.grid()
        ax.legend()

    return fig, axs

## Plot timeseries

In [ ]:
varname_title = {
    "sec": "Surface elevation change of the ice sheets from radar altimetry",
    "sec_err": "Mean standard error of the surface elevation change of the ice sheets from radar altimetry",
    "missing": "Time series of the percentage missing data of the surface elevation change of the ice sheets",
}

for varname, title in varname_title.items():
    fig, axs = plot_timeseries(datasets, varname)
    fig.suptitle(title)

## Plot trends

In [ ]:
fig, axs = plot_trends(pixel_datasets, "sec")
_ = fig.suptitle(
    "Cumulative surface elevation change of the Jakobshavn Glacier and its trends"
)